In [401]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import scipy.special
import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Latex

ALPHA = 0.05

In [5]:
df = pd.read_csv('IB_statistics_data_hw2.9.txt', sep=' ', )
df.head()

,Ex1y,Ex1_group1,Ex1_group2,Ex2y,Ex2_group1,Ex2_group2,Ex2_group3
0,6.602603,1,1,14.649786,1,1,1
1,8.400579,1,1,11.773946,1,1,1
2,9.813724,1,1,12.826394,1,1,1
3,12.981159,1,1,15.567701,1,1,1
4,14.264021,1,1,11.157773,1,1,1


# 1.

Для генерации матрицы плана используем функцию из предыдущего домашнего задания.

## a)

In [72]:
def generate_plan_matrix(sizes):
    r, s = sizes.shape
    pm = np.zeros(shape=(int(sizes.sum()), r + s))
    index1 = [0, 0]           
    index2 = [0, r]
    for i in range(r):
        index2 = [index2[0], r]
        num_of_obs = np.sum(sizes[i])
        pm[index1[0]:index1[0]+num_of_obs, index1[1]] = 1
        index1[0] += num_of_obs
        index1[1] += 1
        for j in range(s):
            num_of_obs = sizes[i, j]
            pm[index2[0]:index2[0]+num_of_obs, index2[1]] =1
            index2[0] += num_of_obs
            index2[1] += 1
    # добавить столбец единичек
    return np.insert(pm, 0, 1, axis=1)

In [391]:
r = df['Ex1_group1'].nunique()
s = df['Ex1_group2'].nunique()
print('Количество разбиений по факторам: ')
print(f'r = {r}; s = {s}')
group_sizes = np.zeros(shape=(r, s))

for g1 in df['Ex1_group1'].unique():
    for g2 in df['Ex1_group2'].unique():
        i, j = g1 - 1, g2 - 2
        group_sizes[i, j] = df.query('(Ex1_group1 == @g1) & (Ex1_group2 == @g2)').shape[0]
print('Размер групп:')
display(group_sizes)

X = generate_plan_matrix(group_sizes.astype(int))

# добавим межфакторное взаимодействие
interf = np.zeros(shape=(df.shape[0], r * s))
column_index = 0
for g1 in df['Ex1_group1'].unique():
    for g2 in df['Ex1_group2'].unique():
        coef = np.zeros(shape=df.shape[0])
        coef[df.query('(Ex1_group1 == @g1) & (Ex1_group2 == @g2)').index] = 1
        interf[:, column_index] = coef
        column_index += 1
        
print('Матрица плана')
X = np.hstack([X, interf])
display(X)

# добавим к матрице плана ограничения - сумма коэффициентов для каждого фактора 
# (и их взаимодействия) равна 0
cond_alpha = [0] + [1] * r + [0] * (s + r * s)
cond_beta = [0] + [0] * r + [1] * s + [0] * (r * s)
cond_interf = [0] * (r * s) + [1] * (r * s)

y = df['Ex1y'].to_numpy()
for cond in (cond_alpha, cond_beta, cond_interf):
    y = np.append(y, 0)
    X = np.insert(X, X.shape[0], cond, axis=0)

# построим мнк-оценки
coef = np.linalg.lstsq(X, y, rcond=None)[0]
print('МНК-оценки с ограничениями:')
display(Latex(fr"$\mu = {round(coef[0], 3)}$"))
display(Latex(r"$\hat \alpha= $"))
display(coef[1:r+1])
display(Latex(r"$\hat \beta = $"))
display(coef[r+1:r+s+1])
display(Latex(r"$\hat {\alpha\beta} = $"))
display(coef[r+s+1:])

Количество разбиений по факторам: 
r = 3; s = 2
Размер групп:


array([[49., 30.],
       [28., 31.],
       [26., 44.]])

Матрица плана


array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

МНК-оценки с ограничениями:


...

array([ 0.75229344,  1.42480934, -2.17710277])

<IPython.core.display.Latex object>

array([-0.32690384,  0.32690384])

<IPython.core.display.Latex object>

array([-1.272905  ,  2.02519844, -1.87673019,  3.30153953,  0.96339756,
       -3.14050033])

($N$ - размер выборки)
1) $H_0: \alpha_1 = \alpha_2 = \alpha_3 = 0$

$$F = \frac{N - rs}{r-1} \cdot \frac{Q_\alpha}{Q_0}\sim \text{F} (r-1, N - rs)$$

In [392]:
N = df.shape[0]

Q0 = np.sum((y - X @ coef)**2)
display(Latex(rf'$Q_0 = {round(Q0, 3)}$'))

coef_H0alpha = coef.copy()
coef_H0alpha[1:r+1] = 0
Qalpha = np.sum((y - X @ coef_H0alpha)**2)
display(Latex(rf'$Q_\alpha = {round(Qalpha, 3)}$'))

distr = sp.stats.f(r - 1, N - r*s)

F = (N - r * s) / (r - 1) * (Qalpha / Q0)
Fcrit = distr.ppf(1 - ALPHA)
pvalue = 1 - distr.cdf(F)

display(Latex(rf"$F = {round(F, 3)}, F_\alpha = {round(Fcrit, 3)}$"))
display(Latex(rf"$p = {round(pvalue, 3)}$"))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

$H_0: \vec \alpha = \vec 0$ отклоняется.

2) $H_0: \vec {\alpha \beta} = \vec 0$
$$F = \frac{N - rs}{(r-1)(s - 1)} \cdot \frac{Q_{\alpha\beta}}{Q_0}\sim \text{F} ((r-1)(s-1), N - rs)$$

In [393]:
coef_H0interf = coef.copy()
coef_H0interf[r+s+1:] = 0

Qab = np.sum((y - X @ coef_H0interf)**2)
display(Latex(r'$Q_{\alpha \beta} = ' + rf'{round(Qab, 3)}$'))

distr = sp.stats.f((r - 1) * (s - 1), N - r*s)

F = (N - r*s) / ((r - 1) * (s - 1)) * (Qab / Q0)
pvalue = 1 - distr.cdf(F)
display(Latex(rf"$F = {round(F, 3)}, F_\alpha = {round(Fcrit, 3)}$"))
display(Latex(rf"$p = {round(pvalue, 3)}$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

$H_0: \vec {\alpha \beta} = \vec 0$ отклоняется.

## b)

In [198]:
# отбросим ограничения
y = y[:-3]
X = X[:-3]

In [193]:
coef

array([13.25207173,  0.75229344,  1.42480934, -2.17710277, -0.32690384,
        0.32690384, -1.272905  ,  2.02519844, -1.87673019,  3.30153953,
        0.96339756, -3.14050033])

In [203]:
B = X.T @ X
Bpinv = np.linalg.pinv(B)
coef_minnorm = Bpinv @ X.T @ y
print('МНК-оценки с минимальной нормой:')
display(Latex(rf"$\mu = {round(coef_minnorm[0], 3)}$"))
display(Latex(r"$\hat \alpha= $"))
display(coef_minnorm[1:r+1])
display(Latex(r"$\hat \beta = $"))
display(coef_minnorm[r+1:r+s+1])
display(Latex(r"$\hat {\alpha\beta} = $"))
display(coef_minnorm[r+s+1:])

МНК-оценки с минимальной нормой:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([2.96097206, 3.63348796, 0.03157585])

<IPython.core.display.Latex object>

array([2.98611409, 3.63992177])

<IPython.core.display.Latex object>

array([-0.16856569,  3.12953775, -0.77239088,  4.40587884,  2.06773687,
       -2.03616102])

In [210]:
labels = ['m'] + [f'a{i+1}' for i in range(r)] + \
    [f'b{i+1}' for i in range(s)] + \
    [f'a{i+1}b{j+1}' for i in range(r) for j in range(s)]
for label, c, cmn in zip(labels, coef, coef_minnorm):
    print(label, c, cmn, sep='\t')

m	13.252071729680662	6.626035864840327
a1	0.7522934373917345	2.960972059005183
a2	1.4248093355157785	3.633487957129224
a3	-2.177102772907517	0.03157584870592778
b1	-0.32690384046787824	2.9861140919522855
b2	0.32690384046788024	3.6399217728880453
a1b1	-1.2729050042366343	-0.16856569342991645
a1b2	2.0251984416283744	3.1295377524350956
a2b1	-1.8767301920137964	-0.7723908812070799
a2b2	3.3015395275295756	4.405878838336304
a3b1	0.9633975568369957	2.0677368676437187
a3b2	-3.1405003297445093	-2.0361610189377886


In [204]:
print(y.mean())

13.481879778648688


Большинство коэффициентов отличаются весьма "сильно" - в частности, $\mu$ при построении оценок минимальной нормы c использованием псевдообратной матрицы очень далёк от среднего значения целевой переменной. Вариант с ограничениями поэтому кажется по-адекватнее.

## c)

Ковариацонная матрица вектора параметров суть (1, 3)-обратная матрица к $X^T X$.

In [359]:
B = X.T @ X
P0 = np.linalg.pinv(B)  # начинаем с псевдообратной матрицы

In [374]:
ndim = B.shape[0]

def loss_function(P):
    # функционал, который требуется минимизировать
    # здесь P - 1,3-обратная матрица к X.T @ X
    np.fill_diagonal(P, 0)
    return np.sum(P**2)

def computeP13(Z):
    # параметризация 1,3 - обратных матриц
    return P0 + (np.eye(ndim) - P0 @ B) @ Z 

def crossover(Z1, Z2):
    # заполняем новую матрицу случайными элементами родителей
    index = np.random.randint(0, 2, size=(ndim, ndim))
    return np.where(index, Z1, Z2)

def mutate(Z, sigma2=0.05):
    noise = np.random.normal(0, sigma2, size=(ndim, ndim))
    return Z + noise


In [375]:
np.random.seed(0)

population_size = 100
N_generations = 100
half = N_generations // 2

best_Z = P0
best_loss = loss_function(P0)
population = [P0 + np.random.normal(0, 1, size=(ndim, ndim)) for _ in range(population_size)]

for i in range(N_generations):
    pinvs = [computeP13(Z) for Z in population]
    scores = [loss_function(pinv) for pinv in pinvs]
    tmp = [(p, s) for p, s in zip(pinvs, scores)]
    tmp.sort(key=lambda x: x[1])
    
    if tmp[0][1] < best_loss:
        best_loss = tmp[0][1]
        best_Z = tmp[0][0]
    
    # из 50% матриц с самым низким значением функционала,
    # будем выбирать родителей с вероятностью, обратно пропорцинальной этому значению
    # остальную половину отбросим
    tmp = tmp[:half]
    
    pinvs = [p for p, s in tmp]
    probs = [1/s for p, s in tmp]
    
    total = sum(probs)  
    probs = [p / total for p in probs]  # нормируем вероятности
    
    new_population = []
    for i in range(half):
        p1, p2 = np.asarray(pinvs)[np.random.choice(half, 2, p=probs)]
        # p1, p2 = np.random.choice(pinvs, size=2, p=probs)
        new_population.append(crossover(p1, p2))
    
    new_population.extend(pinvs)
    for i in range(population_size):
        new_population[i] = mutate(new_population[i])
        
    population = new_population

Что-то у меня по-прежнему выходят странные вещи c параметризацией - свойство 1 отказывается выполняться:

In [376]:
print("Наименьшее значение функционала:")
print(best_loss)
B13 = computeP13(best_Z)

print('Свойство 1:')
print(np.sum(B @ B13 @ B - B))  
print('Свойство 3:')
print(np.sum((B @ B13).T - B @ B13))

Наименьшее значение функционала:
0.004737534109974484
Свойство 1:
-19513.9229393482
Свойство 3:
-7.105427357601002e-15


In [355]:
coef = B13 @ X.T @ y
print('МНК-оценки с ограничениями на элементы ковариационной матрицы:')
display(Latex(fr"$\mu = {round(coef[0], 3)}$"))
display(Latex(r"$\hat \alpha= $"))
display(coef[1:r+1])
display(Latex(r"$\hat \beta = $"))
display(coef[r+1:r+s+1])
display(Latex(r"$\hat {\alpha\beta} = $"))
display(coef[r+s+1:])

МНК-оценки с ограничениями на элементы ковариационной матрицы:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([-10.11197578,  -4.42590042, -11.83843343])

<IPython.core.display.Latex object>

array([-38.05613463, -38.25034004])

<IPython.core.display.Latex object>

array([-27.53482936, -30.32950416, -34.80246673, -32.46825147,
       -25.8041495 , -24.34313037])

Все коэффициенты подозрительно отрицательны...

# 2.

In [402]:
df2 = df[[col for col in df.columns if col.startswith('Ex2')]] 
y = df2['Ex2y'].to_numpy()
_, r, s, t = df2.nunique()
r, s, t

(2, 2, 2)

Матрица плана будет блочной, аналогично случаю с двумя факторами, но с большей "вложенностью". То есть после столбца единичек будут 2 столбца, соответствующие делению по первой группе, и разделяющие матрицу на две "части" - в одной части единицы стоят в 1-ом столбце, в другой - во втором.

Далее будут 2 столбца второго фактора, которые аналогичным образом разделяют каждую из имеющихся "частей" матрицы, и затем эти части снова делятся по 3-ему фактору.

Затем идут 3 группы по 4 столбца, соответствующие парным взаимодействиям. Например, для факторов $\alpha$ и $\beta$ в строке объекта в этих столбцах будет записано $(1, 0, 0, 0)$, если объект принадлежит к группам $\alpha_1$ и $\beta_1$; $(0, 1, 0, 0)$, если к $\alpha_1$ и $\beta_2$; $(0, 0, 1, 0)$, если к $\alpha_2$ и $\beta_1$;  $(0, 0, 0, 1)$ если к $\alpha_2$ и $\beta_2$. Аналогично с остальными парными взамиодействиями.

Затем идёт 8 столбцов, по такому же принципу кодирующих взаимодействие троек.

In [423]:
X = []

for i, row in df2.drop(columns=['Ex2y']).iterrows():
    a, b, c = row - 1  # группы, к которым принадлежит объект из одной строчки таблицы
    
    # интерсепт
    x = [1]
    
    # одиночные факторы
    alpha = [0, 0]
    alpha[a] = 1
    
    beta = [0, 0]
    beta[b] = 1
    
    gamma = [0, 0]
    gamma[c] = 1
    
    # парные взаимодействия - 3 пары, в каждой по 4 варианта
    ab = [0] * 4
    ab[a * 2 + b] = 1
    
    bc = [0] * 4
    bc[b * 2 + c] = 1
    
    ac = [0] * 4
    ac[a * 2 + c] = 1
    
    # тройные взаимодействия - 8 вариантов 
    abc = [0] * 8
    abc[4 * a + 2 * b + c] = 1
    
    x.extend(alpha + beta + gamma + ab + bc + ac + abc)
    X.append(x)
    
X = np.array(X)

In [428]:
B = X.T @ X
beta = np.linalg.pinv(B) @ X.T @ y
labels = ['m', 'a1', 'a2', 'b1', 'b2', 'c1', 'c2'] + \
    [f'a{i}b{j}' for i in (1, 2) for j in (1, 2)] + \
    [f'b{i}c{j}' for i in (1, 2) for j in (1, 2)] + \
    [f'a{i}c{j}' for i in (1, 2) for j in (1, 2)] + \
    [f'a{i}b{j}c{k}' for i in (1, 2) for j in (1, 2) for k in (1, 2)]

for label, b in zip(labels, beta):
    print(label, b, sep='\t')

m	2.7597174983915504
a1	1.177464763936721
a2	1.5822527344548296
b1	1.8969585404673768
b2	0.8627589579241719
c1	1.5134493812179834
c2	1.2462681171735714
a1b1	2.136122191431242
a1b2	-0.9586574274945223
a2b1	-0.23916365096387132
a2b2	1.8214163854186978
b1c1	2.9939320941941525
b1c2	-1.0969735537267793
b2c1	-1.4804827129761702
b2c2	2.343241670900346
a1c1	1.7823681474535862
a1c2	-0.6049033835168622
a2c1	-0.2689187662356021
a2c2	1.8511715006904312
a1b1c1	0.5020937054281371
a1b1c2	1.6340284860031078
a1b2c1	1.2802744420254486
a1b2c2	-2.2389318695199703
a2b1c1	2.4918383887660145
a2b1c2	-2.731002039729886
a2b2c1	-2.7607571550016177
a2b2c2	4.582173540420316


## b)
Попробуем в этот раз другую параметризацию...

In [433]:
B = X.T @ X
P0 = np.linalg.pinv(B)  # начинаем с псевдообратной матрицы

In [435]:
ndim = B.shape[0]
def loss_function(P):
    # функционал, который требуется минимизировать
    # здесь P - 1,3-обратная матрица к X.T @ X
    np.fill_diagonal(P, 0)
    return np.sum(P**2)

def computeP134(Z):
    # параметризация 1,3,4 - обратных матриц
    identity = np.eye(ndim)
    return P0 + (identity - P0 @ B) @ Z @ (identity - B @ P0)
    
def crossover(Z1, Z2):
    # заполняем новую матрицу случайными элементами родителей
    index = np.random.randint(0, 2, size=(ndim, ndim))
    return np.where(index, Z1, Z2)

def mutate(Z, sigma2=0.05):
    noise = np.random.normal(0, sigma2, size=(ndim, ndim))
    return Z + noise

In [436]:
np.random.seed(0)

population_size = 100
N_generations = 100
half = N_generations // 2

best_Z = P0
best_loss = loss_function(P0)
population = [P0 + np.random.normal(0, 1, size=(ndim, ndim)) for _ in range(population_size)]

for i in range(N_generations):
    pinvs = [computeP134(Z) for Z in population]
    scores = [loss_function(pinv) for pinv in pinvs]
    tmp = [(p, s) for p, s in zip(pinvs, scores)]
    tmp.sort(key=lambda x: x[1])
    
    if tmp[0][1] < best_loss:
        best_loss = tmp[0][1]
        best_Z = tmp[0][0]
    
    # из 50% матриц с самым низким значением функционала,
    # будем выбирать родителей с вероятностью, обратно пропорцинальной этому значению
    # остальную половину отбросим
    tmp = tmp[:half]
    
    pinvs = [p for p, s in tmp]
    probs = [1/s for p, s in tmp]
    
    total = sum(probs)  
    probs = [p / total for p in probs]  # нормируем вероятности
    
    new_population = []
    for i in range(half):
        p1, p2 = np.asarray(pinvs)[np.random.choice(half, 2, p=probs)]
        # p1, p2 = np.random.choice(pinvs, size=2, p=probs)
        new_population.append(crossover(p1, p2))
    
    new_population.extend(pinvs)
    for i in range(population_size):
        new_population[i] = mutate(new_population[i])
        
    population = new_population

In [438]:
print("Наименьшее значение функционала:")
print(best_loss)
B13 = computeP13(best_Z)

print('Свойство 1:')
print(np.sum(B @ B13 @ B - B))  
print('Свойство 3:')
print(np.sum((B @ B13).T - B @ B13))   # 

Наименьшее значение функционала:
0.0016563541227072075
Свойство 1:
-13798.692852854516
Свойство 3:
-1.7763568394002505e-15


### c)

По аналогии с двухфакторной моделью будем использовать критерий Фишера. Поменяется только число степеней свободы в суммах в числителе и знаменателе статистики (и соответственно в сомножителе перед дробью).

$N$ - объём выборки, $ddof$ - число степеней свободы; $r, s, t$ - число групп при разбиении по трём факторам соответственно.

$Q_0$ - сумма квадратов остатков полной модели; $ddof = N - r\cdot s \cdot t$;

$Q_\alpha$ - усечённая модель для проверки $H_0: \vec \alpha = \vec 0$; $ddof = r - 1$ (аналогично для других факторов)


$Q_{\alpha\beta}$ - усечённая модель для проверки $H_0: \vec {\alpha\beta} = \vec 0$; $ddof = (r - 1)(s-1)$


$Q_{\alpha\beta\gamma}$ - усечённая модель для проверки $H_0: \vec {\alpha\beta\gamma} = \vec 0$; $ddof = (r - 1)(s-1)(t-1)$

Если нужно проверить несколько гипотез (допустим о незначимости первого фактора и взаимодействия между вторым и третьем, то можно пересчитать число степеней свободы в суммах, либо проверить эти гипотезы по-отдельности и воспользоваться поправкой.
